In [26]:
import time

eps = 0.1
minpts = 10

csv_file = '/home/msbd5003/Desktop/data_partition_8p.csv'
numPartition = 8  

start = time.time()

raw_data = sc.textFile(csv_file,numPartition)
raw_data = raw_data.map(lambda p: p.split(","))
header = raw_data.first()
raw_data = raw_data.filter(lambda p: p[0] != header[0])\
                   .map(lambda p: p[1::])\
                   .map(lambda p: [p[0],[[p[1:-1],0],False]])
distrib_record = raw_data.partitionBy(numPartition).map(lambda p: list(p))

def serial_DBScan(data):

    clusters = []

    def getNeighbors(point, eps):
        wait_list = []
        for item in data:
            if (point[1][0][0][0] != item[1][0][0][0]) & ((float(point[1][0][0][1]) - float(item[1][0][0][1]))**2 + (float(point[1][0][0][2]) - float(item[1][0][0][2]))**2 <= eps**2):
                wait_list.append(item[1][0][0][0])
        return wait_list

    def expandCluster(point, N, C, eps, minpts):
        point[1][0][1] = 1
        C.append(point)
        for item_id in N:
            for temp_item in data:
                if item_id == temp_item[1][0][0][0]:
                    item = temp_item
                    break
            item[1][1] = True
            my_N = getNeighbors(item, eps)
            if len(my_N) > minpts:
                N = list(set(N + my_N))
            if item[1][0][1] == 0:
                item[1][0][1] = 1
                C.append(item)

    for point in data:
        if point[1][1] == False:
            point[1][1] = True
            N = getNeighbors(point, eps)
            if len(N) < minpts:
                point[1][0][1] = -1
            else:
                new_cluster = []
                clusters.append(new_cluster)
                expandCluster(point, N, new_cluster, eps, minpts)

    return clusters

def DBScan(it):
    data = []

    for item in it:
        data.append(item)

    clusters = serial_DBScan(data)
    return clusters

distrib_partial_clusters = distrib_record.mapPartitions(DBScan)
partial_clusters = distrib_partial_clusters.collect()

partial_clusters_clean = []

for i in range(len(partial_clusters)):
    data_id = sc.parallelize(partial_clusters[i])\
                .map(lambda p: int(float(p[1][0][0][0])))\
                .collect()
    partial_clusters_clean.append(data_id)

result = []

for cluster1 in partial_clusters_clean:
    while True:
        previous_len = len(cluster1)
        for cluster2 in partial_clusters_clean:
            if len(set(cluster1).intersection(set(cluster2))) != 0:
                cluster1 = list(set(cluster1).union(set(cluster2)))
                cluster1 = sorted(cluster1)
        if len(cluster1) == previous_len:
            break
    if not cluster1 in result:
        result.append(cluster1)

end = time.time()
spend_time = end - start

print str(len(result)) + ' clusters'
print 'spend time ' + str(spend_time) + 's'
for i in range(len(result)):
    print len(result[i])
    print result[i]

12 clusters
spend time 2.7827129364s
10
[86, 152, 198, 430, 445, 538, 600, 629, 661, 732]
8
[258, 355, 448, 477, 529, 562, 590, 671]
9
[30, 95, 200, 509, 511, 535, 566, 740, 747]
8
[119, 137, 217, 293, 401, 537, 595, 689]
9
[48, 222, 288, 326, 444, 574, 579, 691, 703]
20
[11, 12, 34, 70, 75, 97, 142, 169, 196, 206, 216, 325, 356, 361, 425, 589, 639, 657, 727, 743]
12
[15, 27, 111, 121, 235, 289, 339, 347, 386, 436, 447, 586]
12
[41, 62, 168, 297, 388, 534, 549, 554, 611, 640, 641, 735]
12
[16, 53, 83, 192, 373, 435, 440, 517, 533, 683, 684, 713]
10
[42, 232, 322, 370, 461, 488, 515, 561, 698, 730]
6
[35, 102, 103, 143, 344, 522]
13
[19, 68, 76, 84, 123, 162, 218, 245, 247, 300, 335, 384, 624]

# output process

In [4]:
import os
import time
import pandas as pd

In [5]:
records = pd.DataFrame(columns=['file','second'])
files = os.listdir('/home/msbd5003/Desktop/data_all/')[:5]
for f in files:

    csv_file = '/home/msbd5003/Desktop/data_all/'+f
    print(csv_file)

/home/msbd5003/Desktop/data_all/data_partition_2p_120row.csv
/home/msbd5003/Desktop/data_all/data_partition_8p_340row.csv
/home/msbd5003/Desktop/data_all/data_partition_5p_400row.csv
/home/msbd5003/Desktop/data_all/data_partition_8p_480row.csv
/home/msbd5003/Desktop/data_all/data_partition_6p_600row.csv


In [8]:
records = pd.DataFrame(columns=['file','clusters_num','second'])
files = os.listdir('/home/msbd5003/Desktop/data_all/')
counter = 0
counter_all = len(files)

eps = 0.1
minpts = 3

for f in files:
    counter += 1
    csv_file = '/home/msbd5003/Desktop/data_all/'+f 
    numPartition = int(f.split('_')[2][0])    
    print csv_file,str(numPartition)+'p ',str(counter)+'/'+str(counter_all)
    
    start = time.time()
    
    raw_data = sc.textFile(csv_file,numPartition)
    raw_data = raw_data.map(lambda p: p.split(","))
    header = raw_data.first()
    raw_data = raw_data.filter(lambda p: p[0] != header[0])\
                       .map(lambda p: p[1::])\
                       .map(lambda p: [p[0],[[p[1:-1],0],False]])
    distrib_record = raw_data.partitionBy(numPartition).map(lambda p: list(p))

    def serial_DBScan(data):

        clusters = []

        def getNeighbors(point, eps):
            wait_list = []
            for item in data:
                if (point[1][0][0][0] != item[1][0][0][0]) & ((float(point[1][0][0][1]) - float(item[1][0][0][1]))**2 + (float(point[1][0][0][2]) - float(item[1][0][0][2]))**2 <= eps**2):
                    wait_list.append(item[1][0][0][0])
            return wait_list

        def expandCluster(point, N, C, eps, minpts):
            point[1][0][1] = 1
            C.append(point)
            for item_id in N:
                for temp_item in data:
                    if item_id == temp_item[1][0][0][0]:
                        item = temp_item
                        break
                item[1][1] = True
                my_N = getNeighbors(item, eps)
                if len(my_N) > minpts:
                    N = list(set(N + my_N))
                if item[1][0][1] == 0:
                    item[1][0][1] = 1
                    C.append(item)

        for point in data:
            if point[1][1] == False:
                point[1][1] = True
                N = getNeighbors(point, eps)
                if len(N) < minpts:
                    point[1][0][1] = -1
                else:
                    new_cluster = []
                    clusters.append(new_cluster)
                    expandCluster(point, N, new_cluster, eps, minpts)

        return clusters

    def DBScan(it):
        data = []

        for item in it:
            data.append(item)

        clusters = serial_DBScan(data)
        return clusters

    distrib_partial_clusters = distrib_record.mapPartitions(DBScan)
    partial_clusters = distrib_partial_clusters.collect()

    partial_clusters_clean = []

    for i in range(len(partial_clusters)):
        data_id = sc.parallelize(partial_clusters[i])\
                    .map(lambda p: int(float(p[1][0][0][0])))\
                    .collect()
        partial_clusters_clean.append(data_id)

    result = []

    for cluster1 in partial_clusters_clean:
        while True:
            previous_len = len(cluster1)
            for cluster2 in partial_clusters_clean:
                if len(set(cluster1).intersection(set(cluster2))) != 0:
                    cluster1 = list(set(cluster1).union(set(cluster2)))
                    cluster1 = sorted(cluster1)
            if len(cluster1) == previous_len:
                break
        if not cluster1 in result:
            result.append(cluster1)

    end = time.time()
    spend_time = end - start

    print str(len(result)) + ' clusters'
    print 'spend time ' + str(spend_time) + 's'
    #for i in range(len(result)):
        #print len(result[i])
        #print result[i]
    
    
    df = pd.DataFrame(data={'file':[csv_file],'clusters_num':[len(result)],'second':[spend_time]})
    records = records.append(df)
records.to_csv('experiment.csv')

/home/msbd5003/Desktop/data_all/data_partition_2p_120row.csv 2p  1/296
8 clusters
spend time 0.916922807693s
/home/msbd5003/Desktop/data_all/data_partition_8p_340row.csv 8p  2/296
68 clusters
spend time 6.72507715225s
/home/msbd5003/Desktop/data_all/data_partition_5p_400row.csv 5p  3/296
94 clusters
spend time 9.4933218956s
/home/msbd5003/Desktop/data_all/data_partition_8p_480row.csv 8p  4/296
84 clusters
spend time 8.80816411972s
/home/msbd5003/Desktop/data_all/data_partition_6p_600row.csv 6p  5/296
109 clusters
spend time 13.5504279137s
/home/msbd5003/Desktop/data_all/data_partition_3p_120row.csv 3p  6/296
12 clusters
spend time 1.15907406807s
/home/msbd5003/Desktop/data_all/data_partition_8p_240row.csv 8p  7/296
51 clusters
spend time 5.1798350811s
/home/msbd5003/Desktop/data_all/data_partition_3p_140row.csv 3p  8/296
20 clusters
spend time 1.69301605225s
/home/msbd5003/Desktop/data_all/data_partition_4p_720row.csv 4p  9/296
108 clusters
spend time 8.91328120232s
/home/msbd5003/Desk

92 clusters
spend time 7.34476089478s
/home/msbd5003/Desktop/data_all/data_partition_4p_200row.csv 4p  76/296
49 clusters
spend time 3.62361097336s
/home/msbd5003/Desktop/data_all/data_partition_6p_720row.csv 6p  77/296
109 clusters
spend time 13.8214437962s
/home/msbd5003/Desktop/data_all/data_partition_4p_540row.csv 4p  78/296
94 clusters
spend time 7.72699189186s
/home/msbd5003/Desktop/data_all/data_partition_6p_80row.csv 6p  79/296
8 clusters
spend time 1.32450795174s
/home/msbd5003/Desktop/data_all/data_partition_5p_420row.csv 5p  80/296
96 clusters
spend time 9.58791613579s
/home/msbd5003/Desktop/data_all/data_partition_7p_160row.csv 7p  81/296
39 clusters
spend time 3.96129202843s
/home/msbd5003/Desktop/data_all/data_partition_6p_240row.csv 6p  82/296
62 clusters
spend time 6.95032787323s
/home/msbd5003/Desktop/data_all/data_partition_7p_300row.csv 7p  83/296
76 clusters
spend time 7.55700612068s
/home/msbd5003/Desktop/data_all/data_partition_3p_420row.csv 3p  84/296
111 cluster

1 clusters
spend time 0.275804042816s
/home/msbd5003/Desktop/data_all/data_partition_4p_440row.csv 4p  151/296
85 clusters
spend time 6.46791386604s
/home/msbd5003/Desktop/data_all/data_partition_6p_340row.csv 6p  152/296
79 clusters
spend time 8.96858692169s
/home/msbd5003/Desktop/data_all/data_partition_3p_180row.csv 3p  153/296
37 clusters
spend time 3.11388492584s
/home/msbd5003/Desktop/data_all/data_partition_5p_380row.csv 5p  154/296
89 clusters
spend time 8.76848006248s
/home/msbd5003/Desktop/data_all/data_partition_2p_200row.csv 2p  155/296
31 clusters
spend time 2.24619698524s
/home/msbd5003/Desktop/data_all/data_partition_8p_80row.csv 8p  156/296
14 clusters
spend time 1.92724204063s
/home/msbd5003/Desktop/data_all/data_partition_3p_580row.csv 3p  157/296
122 clusters
spend time 8.87387895584s
/home/msbd5003/Desktop/data_all/data_partition_8p_700row.csv 8p  158/296
102 clusters
spend time 11.1882219315s
/home/msbd5003/Desktop/data_all/data_partition_5p_700row.csv 5p  159/296


2 clusters
spend time 0.887909173965s
/home/msbd5003/Desktop/data_all/data_partition_8p_20row.csv 8p  225/296
2 clusters
spend time 0.987837076187s
/home/msbd5003/Desktop/data_all/data_partition_5p_580row.csv 5p  226/296
117 clusters
spend time 12.0689668655s
/home/msbd5003/Desktop/data_all/data_partition_2p_700row.csv 2p  227/296
116 clusters
spend time 8.40105485916s
/home/msbd5003/Desktop/data_all/data_partition_8p_380row.csv 8p  228/296
75 clusters
spend time 7.28076100349s
/home/msbd5003/Desktop/data_all/data_partition_3p_200row.csv 3p  229/296
41 clusters
spend time 3.07309484482s
/home/msbd5003/Desktop/data_all/data_partition_2p_560row.csv 2p  230/296
102 clusters
spend time 7.09953689575s
/home/msbd5003/Desktop/data_all/data_partition_6p_700row.csv 6p  231/296
109 clusters
spend time 13.8403019905s
/home/msbd5003/Desktop/data_all/data_partition_2p_640row.csv 2p  232/296
108 clusters
spend time 7.50882291794s
/home/msbd5003/Desktop/data_all/data_partition_2p_540row.csv 2p  233/2

In [7]:
records.to_csv('experiment.csv')